In [1]:
%pip install networkx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import os
import subprocess
import re
import contextlib
import io
import sys
import networkx as nx
import csv
import time
import Parameters
import GraphS
import Pattern

subdue_example_path = 'dataset.csv'
tolerance_pct = 0.1  # mainly due to non-deterministic nature of the algorithm

In [12]:
def subdue_csv_to_undirected_nx_graph(subdue_json_path):
    """WARNING: ignores directed edges and timestamp on purpose"""

    with open(subdue_json_path, 'r') as subdue_json_file:
        subdue_format = csv.load(subdue_json_file)

    graph = nx.Graph()
    for vertex_or_edge in subdue_format:
        if list(vertex_or_edge.keys()) == ['vertex']:
            node_attributes_loop = vertex_or_edge['vertex']['attributes']
            graph.add_node(
                vertex_or_edge['vertex']['id'],
                **node_attributes_loop,
            )
        elif list(vertex_or_edge.keys()) == ['edge']:
            edge_attributes_loop = vertex_or_edge['edge']['attributes']
            graph.add_edge(
                u_of_edge=vertex_or_edge['edge']['source'],
                v_of_edge=vertex_or_edge['edge']['target'],
                **edge_attributes_loop,
            )
        else:
            raise ValueError('Invalid entry type')
    return graph


In [22]:
def GetInitialPatterns(parameters, graph):
    """Returns list of single-edge, evaluated patterns in given graph with more than one instance."""
    initialPatternList = []
    # Create a graph and an instance for each edge
    edgeGraphInstancePairs = []
    for edge in graph.edges.values():
        graph1 = GraphS.CreateGraphFromEdge(edge)
        if parameters.temporal:
            graph1.TemporalOrder()
        instance1 = Pattern.CreateInstanceFromEdge(edge)
        edgeGraphInstancePairs.append((graph1,instance1))
    while edgeGraphInstancePairs:
        edgePair1 = edgeGraphInstancePairs.pop(0)
        graph1 = edgePair1[0]
        instance1 = edgePair1[1]
        pattern = Pattern.Pattern()
        pattern.definition = graph1
        pattern.instances.append(instance1)
        nonmatchingEdgePairs = []
        for edgePair2 in edgeGraphInstancePairs:
            graph2 = edgePair2[0]
            instance2 = edgePair2[1]
            if GraphS.GraphMatch(graph1,graph2) and (not Pattern.InstancesOverlap(parameters.overlap, pattern.instances, instance2)):
                pattern.instances.append(instance2)
            else:
                nonmatchingEdgePairs.append(edgePair2)
        if len(pattern.instances) > 1:
            pattern.evaluate(graph)
            initialPatternList.append(pattern)
        edgeGraphInstancePairs = nonmatchingEdgePairs
    return initialPatternList

DEBUGFLAG = False

def DiscoverPatterns(parameters, graph):
    """The main discovery loop. Finds and returns best patterns in given graph."""
    patternCount = 0
    # get initial one-edge patterns
    parentPatternList = GetInitialPatterns(parameters, graph)
    if DEBUGFLAG:
        print("Initial patterns (" + str(len(parentPatternList)) + "):")
        for pattern in parentPatternList:
            pattern.print_pattern('  ')
    discoveredPatternList = []
    while ((patternCount < parameters.limit) and parentPatternList):
        print(str(int(parameters.limit - patternCount)) + " patterns left", flush=True)
        childPatternList = []
        # extend each pattern in parent list (***** todo: in parallel)
        while (parentPatternList):
            parentPattern = parentPatternList.pop(0)
            if ((len(parentPattern.instances) > 1) and (patternCount < parameters.limit)):
                patternCount += 1
                extendedPatternList = Pattern.ExtendPattern(parameters, parentPattern)
                while (extendedPatternList):
                    extendedPattern = extendedPatternList.pop(0)
                    if DEBUGFLAG:
                        print("Extended Pattern:")
                        extendedPattern.print_pattern('  ')
                    if (len(extendedPattern.definition.edges) <= parameters.maxSize):
                        # evaluate each extension and add to child list
                        extendedPattern.evaluate(graph)
                        if ((not parameters.prune) or (extendedPattern.value >= parentPattern.value)):
                            Pattern.PatternListInsert(extendedPattern, childPatternList, parameters.beamWidth, parameters.valueBased)
            # add parent pattern to final discovered list
            if (len(parentPattern.definition.edges) >= parameters.minSize):
                Pattern.PatternListInsert(parentPattern, discoveredPatternList, parameters.numBest, False) # valueBased = False
        parentPatternList = childPatternList
        if not parentPatternList:
            print("No more patterns to consider", flush=True)
    # insert any remaining patterns in parent list on to discovered list
    while (parentPatternList):
        parentPattern = parentPatternList.pop(0)
        if (len(parentPattern.definition.edges) >= parameters.minSize):
            Pattern.PatternListInsert(parentPattern, discoveredPatternList, parameters.numBest, False) # valueBased = False
    return discoveredPatternList

def Subdue(parameters, graph):
    """
    Top-level function for Subdue that discovers best pattern in graph.
    Optionally, Subdue can then compress the graph with the best pattern, and iterate.

    :param graph: instance of Subdue.Graph
    :param parameters: instance of Subdue.Parameters
    :return: patterns for each iteration -- a list of iterations each containing discovered patterns.
    """
    startTime = time.time()
    iteration = 1
    done = False
    patterns = list()
    while ((iteration <= parameters.iterations) and (not done)):
        iterationStartTime = time.time()
        if (iteration > 1):
            print("----- Iteration " + str(iteration) + " -----\n")
        print("Graph: " + str(len(graph.vertices)) + " vertices, " + str(len(graph.edges)) + " edges")
        patternList = DiscoverPatterns(parameters, graph)
        if (not patternList):
            done = True
            print("No patterns found.\n")
        else:
            patterns.append(patternList)
            print("\nBest " + str(len(patternList)) + " patterns:\n")
            for pattern in patternList:
                pattern.print_pattern('  ')
                print("")
            # write machine-readable output, if requested
            if (parameters.writePattern):
                outputFileName = parameters.outputFileName + "-pattern-" + str(iteration) + ".json"
                patternList[0].definition.write_to_file(outputFileName)
            if (parameters.writeInstances):
                outputFileName = parameters.outputFileName + "-instances-" + str(iteration) + ".json"
                patternList[0].write_instances_to_file(outputFileName)
            if ((iteration < parameters.iterations) or (parameters.writeCompressed)):
                graph.Compress(iteration, patternList[0])
            if (iteration < parameters.iterations):
                # consider another iteration
                if (len(graph.edges) == 0):
                    done = True
                    print("Ending iterations - graph fully compressed.\n")
            if ((iteration == parameters.iterations) and (parameters.writeCompressed)):
                outputFileName = parameters.outputFileName + "-compressed-" + str(iteration) + ".json"
                graph.write_to_file(outputFileName)
        if (parameters.iterations > 1):
             iterationEndTime = time.time()
             print("Elapsed time for iteration " + str(iteration) + " = " + str(iterationEndTime - iterationStartTime) + " seconds.\n")
        iteration += 1
    endTime = time.time()
    print("SUBDUE done. Elapsed time = " + str(endTime - startTime) + " seconds\n")
    return patterns

In [23]:
def ReadGraph(inputFileName):
    """Read graph from given filename."""
    inputfile = open(inputFileName)
    csvGraphArray = csv.load(inputfile)
    graph = GraphS.Graph()
    graph.load_from_csv(csvGraphArray)
    inputfile.close()
    return graph

def main_with_path(file_path):
    """
    Imitates the cli call to subdue with default parameters and a `file_path`
    """
    parameters = Parameters.Parameters()
    # parameters.set_parameters(sys.argv)
    graph = ReadGraph(file_path)
    # outputFileName = parameters.outputFileName + ".dot"
    # graph.write_to_dot(outputFileName)
    parameters.set_defaults_for_graph(graph)
    Subdue(parameters, graph)

Subdue

- Input: Là một đồ thị đơn hoặc tập các đồ thị, các đồ thị có thể được gán nhãn hoặc không.
- Output: Danh sách các đồ thị
 Các đồ thị có thể được gắn nhãn hoặc không. Subdue đưa ra các cấu trúc con nén tốt nhất tập dữ liệu đầu vào theo nguyên tắc Độ Dài Mô Tả Tối Thiểu (MDL). Subdue thực hiện tìm kiếm dầm bị hạn chế về tính toán, bắt đầu từ các cấu trúc con bao gồm tất cả các đỉnh có nhãn duy nhất. Các cấu trúc con được mở rộng bằng một đỉnh và một cạnh hoặc một cạnh theo mọi cách có thể, được hướng dẫn bởi các đồ thị ví dụ, để tạo ra các cấu trúc con ứng viên. Subdue duy trì các trường hợp của các cấu trúc con trong các ví dụ và sử dụng đồng cấu đồ thị để xác định các trường hợp của cấu trúc con ứng viên trong các ví dụ. Các cấu trúc con sau đó được đánh giá dựa trên mức độ nén tốt của chúng đối với Độ Dài Mô Tả (DL) của tập dữ liệu. DL của tập dữ liệu đầu vào G sử dụng cấu trúc con S có thể được tính toán bằng công thức sau: I(S) + I(G|S)


In [41]:
import networkx as nx
import matplotlib.pyplot as plt
import csv

def parse_edge(edge_str):
    """Parses an edge string into a tuple."""
    return tuple(map(int, edge_str.strip('()').split(',')))

def read_graphs_from_csv(file_path):
    """Reads multiple graphs from a CSV file where each row represents a graph."""
    graphs = []
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        for row in reader:
            graph = nx.Graph()
            edges = [parse_edge(edge) for edge in row]
            graph.add_edges_from(edges)
            graphs.append(graph)
    return graphs

def subdue(graph, beam_width, max_best, max_sub_size, limit):
    parent_list = []
    child_list = []
    best_list = []
    processed_subs = 0

    for vertex in graph.nodes():
        substructure = create_substructure(vertex)
        parent_list.append(substructure)

    while processed_subs <= limit and parent_list:
        while parent_list:
            parent = parent_list.pop(0)
            extended_instances = extend_instances(parent, graph)

            for child in extended_instances:
                if size_of(child) < max_sub_size:
                    evaluate_child(child)
                    insert_in_order(child_list, child, beam_width)

            processed_subs += 1
            insert_in_order(best_list, parent, max_best)

        parent_list, child_list = child_list, []

    return best_list

def create_substructure(vertex):
    return Substructure(vertex_label=vertex)

def extend_instances(parent, graph):
    extended = []
    for vertex in graph.neighbors(parent.vertex_label):
        new_substructure = Substructure(vertex_label=vertex, parent=parent)
        extended.append(new_substructure)
    return extended

def size_of(substructure):
    size = 0
    current = substructure
    while current is not None:
        size += 1
        current = current.parent
    return size

def evaluate_child(child):
    child.value = size_of(child)

def insert_in_order(lst, item, max_length):
    lst.append(item)
    lst.sort(key=lambda x: x.value, reverse=True)
    if len(lst) > max_length:
        lst.pop()

class Substructure:
    def __init__(self, vertex_label, parent=None):
        self.vertex_label = vertex_label
        self.parent = parent
        self.value = 0

def visualize_graph(graph, best_substructures=None):
    """Visualizes the graph with optional highlighting of best substructures."""
    pos = nx.spring_layout(graph)  # Use spring layout for node positioning

    # Draw the graph
    plt.figure(figsize=(12, 8))
    nx.draw(graph, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=500, font_size=10, font_color='black')

    # Highlight best substructures if provided
    if best_substructures:
        for substructure in best_substructures:
            nodes = set()
            current = substructure
            while current is not None:
                nodes.add(current.vertex_label)
                current = current.parent
            subgraph = graph.subgraph(nodes)
            nx.draw(subgraph, pos, with_labels=True, edge_color='red', node_color='lightgreen', node_size=700, font_size=12, font_color='black')

    plt.title('Graph Visualization')
    plt.show()

# Example usage:
file_path = 'dataset.csv'
graphs = read_graphs_from_csv(file_path)
beam_width = 5
max_best = 5
max_sub_size = 5
limit = 100

for i, graph in enumerate(graphs):
    print(f"Processing graph {i+1}")
    best_substructures = subdue(graph, beam_width, max_best, max_sub_size, limit)
    print(f"Best substructures for graph {i+1}:")
    for substructure in best_substructures:
        print(f"Substructure with value {substructure.value}: {substructure.vertex_label}")

    # Visualize the graph and best substructures
    visualize_graph(graph, best_substructures)


ValueError: invalid literal for int() with base 10: 'graphs'

In [38]:
file_path = 'dataset.csv'
graphs = read_graphs_from_csv(file_path)
for i, graph in enumerate(graphs):
    print(f"Graph {i+1}:")
    print("Nodes:", graph.nodes())
    print("Edges:", graph.edges())

In [25]:
def unify_output_text(text):
    text = re.sub(r'Elapsed time = [0-9]*.[0-9]*', 'Elapsed time = REMOVED', text)
    text = re.sub(r'timestamp=[0-9]', 'timestamp=0', text)
    text = re.sub(r'timestamp=0[0-9]', 'timestamp=0', text)
    text = re.sub('edge \"[0-9]*\"', 'edge', text)
    text = re.sub('edge \"[0-9]*-[0-9]*\"', 'edge', text)
    return text

In [29]:
def nx_subdue(
    graph,
    node_attributes=None,
    edge_attributes=None,
    verbose=False,
    **subdue_parameters
):
    """
    :param graph: networkx.Graph
    :param node_attributes: (Default: None)   -- attributes on the nodes to use for pattern matching, use `None` for all
    :param edge_attributes: (Default: None)   -- attributes on the edges to use for pattern matching, use `None` for all
    :param verbose: (Default: False)          -- if True, print progress, as well as report each found pattern

    :param beamWidth: (Default: 4)            -- Number of patterns to retain after each expansion of previous patterns; based on value.
    :param iterations: (Default: 1)           -- Iterations of Subdue's discovery process. If more than 1, Subdue compresses graph with best pattern before next run. If 0, then run until no more compression (i.e., set to |E|).
    :param limit: (Default: 0)                -- Number of patterns considered; default (0) is |E|/2.
    :param maxSize: (Default: 0)              -- Maximum size (#edges) of a pattern; default (0) is |E|/2.
    :param minSize: (Default: 1)              -- Minimum size (#edges) of a pattern; default is 1.
    :param numBest: (Default: 3)              -- Number of best patterns to report at end; default is 3.
    :param overlap: (Defaul: none)            -- Extent that pattern instances can overlap (none, vertex, edge)
    :param prune: (Default: False)            -- Remove any patterns that are worse than their parent.
    :param valueBased: (Default: False)       -- Retain all patterns with the top beam best values.
    :param temporal: (Default: False)         -- Discover static (False) or temporal (True) patterns

    :return: list of patterns, where each pattern is a list of pattern instances, with an instance being a dictionary
    containing 
        `nodes` -- list of IDs, which can be used with `networkx.Graph.subgraph()`
        `edges` -- list of tuples (id_from, id_to), which can be used with `networkx.Graph.edge_subgraph()`
    
    For `iterations`>1 the the list is split by iterations, and some patterns will contain node IDs not present in
    the original graph, e.g. `PATTERN-X-Y`, such node ID refers to a previously compressed pattern, and it can be 
    accessed as `output[X-1][0][Y]`.

    """
    parameters = Parameters.Parameters()
    if len(subdue_parameters) > 0:
        parameters.set_parameters_from_kwargs(**subdue_parameters)
    subdue_graph = GraphS.Graph()
    subdue_graph.load_from_networkx(graph, node_attributes, edge_attributes)
    parameters.set_defaults_for_graph(subdue_graph)
    if verbose:
        iterations = Subdue(parameters, subdue_graph)
    else:
        with contextlib.redirect_stdout(None):
            iterations = Subdue(parameters, subdue_graph)
    iterations = unwrap_output(iterations)
    if parameters.iterations == 1:
        if len(iterations) == 0:
            return None
        return iterations[0]
    else:
        return iterations

def unwrap_output(iterations):
    """
    Subroutine of `nx_Subdue` -- unwraps the standard Subdue output into pure python objects compatible with networkx
    """
    out = list()
    for iteration in iterations:
        iter_out = list()
        for pattern in iteration:
            pattern_out = list()
            for instance in pattern.instances:
                pattern_out.append({
                    'nodes': [vertex.id for vertex in instance.vertices],
                    'edges': [(edge.source.id, edge.target.id) for edge in instance.edges]
                })
            iter_out.append(pattern_out)
        out.append(iter_out)
    return out
    
# normal run
capture_prints = io.StringIO()
with contextlib.redirect_stdout(capture_prints):
    main_with_path(subdue_example_path)
prints_main = capture_prints.getvalue()


# use networkx graph as an input
subdue_example_graph = subdue_csv_to_undirected_nx_graph(subdue_example_path)
capture_prints = io.StringIO()
with contextlib.redirect_stdout(capture_prints):
    nx_subdue(graph=subdue_example_graph, verbose=True)
prints_nx_subdue = capture_prints.getvalue()


# compare
prints_main = unify_output_text(prints_main)
prints_nx_subdue = unify_output_text(prints_nx_subdue)

assert (
    (
        len(set(prints_nx_subdue.split('\n')) - set(prints_main.split('\n')))
        +
        len(set(prints_main.split('\n')) - set(prints_nx_subdue.split('\n')))
    )
    /
    len(prints_main.split('\n'))
) < tolerance_pct

def main():
    print("SUBDUE v1.4 (python)\n")
    parameters = Parameters.Parameters()
    parameters.set_parameters(sys.argv)
    graph = ReadGraph(parameters.inputFileName)
    #outputFileName = parameters.outputFileName + ".dot"
    #graph.write_to_dot(outputFileName)
    parameters.set_defaults_for_graph(graph)
    parameters.print()
    Subdue(parameters, graph)

if __name__ == "__main__":
    main()

AttributeError: module 'Parameters' has no attribute 'Parameters'